## Train VAE and AE on SVHN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision.datasets import MNIST,CelebA
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchvision import transforms
import os
from tqdm.notebook import tqdm
from AE import *
import numpy as np

import torch.nn.functional as F

In [5]:
def get_data_loader(dataset, batch_size, cuda=False):
    return DataLoader(
        dataset, batch_size=batch_size, shuffle=True,
        **({'num_workers': 10, 'pin_memory': True} if cuda else {})
    )

In [6]:
# Train-Function AE
def train_AE(model, dataset, epochs=20, 
         batch_size=32, img_size=32,
         lr=0.0005, weight_decay=1e-3,
         device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
    
    # intitialize 
    resume=False  # set true to continue training
    if resume:
        model.load_state_dict(torch.load('./___________.pth',map_location=torch.device('cuda')))
        model.to(torch.device("cuda:0"))
        epoch_start=xxx # set epoch counter to resume
        print('model loaded')
        resume=False
    else:
        epoch_start = 0
        
    
    model = model.to(device)    
    criterion = torch.nn.BCELoss(size_average=False) #input range [0,1]
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    if not os.path.exists(output_folder):
            os.makedirs(output_folder)
    data_root = './' 
    
    # iterate over epochs
    for epoch in range(epoch_start, epochs + 1):
        model.train()
        data_loader=get_data_loader(dataset, batch_size=batch_size,cuda=True)
        print("Epoch {}".format(epoch))
        for batch_index, (x, _) in tqdm(enumerate(data_loader, 1)):
            iteration = (epoch - 1) * (len(dataset) // batch_size) + batch_index
            x = Variable(x).to(device)
            x_reconstructed = model(x)
            loss = criterion(x_reconstructed, x)        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

  
        if (epoch) % 2 == 0:
            torch.save(model.state_dict(), './ae_SVHN_{}.pth'.format(epoch))
        print("epoch : {}/{}, recon loss = {:.8f}".format(epoch , epochs, loss))          

In [7]:
# Train-Function VAE
def train_VAE(model, dataset, epochs=20, 
         batch_size=32, img_size=32,
         lr=1e-3, weight_decay=1e-3,
         device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
    
    
    # initialize
    resume=False # set true to continue training
    if resume:
         model.load_state_dict(torch.load('./_________.pth',map_location=torch.device('cuda')))  # insert model name
        model.to(torch.device("cuda:0"))
        epoch_start=xxx  # set epoch counter to resume
        print('model loaded')
        resume=False
    else:
        epoch_start = 0
    data_loader=get_data_loader(dataset, batch_size=batch_size,cuda=True)
    

    model = model.to(device)    
    criterion = torch.nn.BCELoss(size_average=False)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    #iterate over epochs
    for epoch in range(epochs):
        model.train()
        print("Epoch {}".format(epoch))
        for batch_index, (x, _) in tqdm(enumerate(data_loader, 1)):
            x = x.to(device)
            (mean, logvar), x_reconstructed = model(x)
            penalization_loss = model.kl_divergence_loss(mean, logvar)
            reconstruction_loss = criterion(x_reconstructed, x) / x.size(0)
            loss = reconstruction_loss + penalization_loss
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        print("epoch : {}/{}, recon loss = {:.8f}".format(epoch + 1, epochs, loss))
        if epoch % 2 == 0:
            torch.save(model.state_dict(), './vae_SVHN_{}.pth'.format(epoch))
            

### Training

In [27]:
#load data
path1 = %pwd
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

trans_SVHN =transforms.ToTensor()
dataset=SVHN(path1,split="train", transform=trans_SVHN, download=False)

cuda


#### Train VAE

In [ ]:
# load model
model = vae_SVHN(image_size=32, hidden_dim=20, z_size=20, device='cuda')

In [ ]:
import timeit
import utils
start = timeit.default_timer()
epochs = 200
train_VAE(model, dataset, epochs, batch_size=32, img_size=32,lr=0.0005, weight_decay=1e-3)
end = timeit.default_timer()
print(end-start)

#### Train AE

In [28]:
#load model
model = ae_SVHN(image_size=32, hidden_dim=20, z_size=20, device='cuda')

In [ ]:
import timeit
import utils
start = timeit.default_timer()
epochs = 200
train_AE(model, dataset, epochs,   batch_size=32, img_size=32, lr=0.0005, weight_decay=1e-3,device='cuda')
end = timeit.default_timer()
print(end-start)